In [1]:
import numpy as np
import math
import operator
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
data = pd.read_csv('泰坦尼克号数据.csv')
data = data.replace('',np.NaN)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
data = data.drop(columns='Cabin')
data = data.drop(columns='PassengerId')
data = data.drop(columns='Name')
data = data.drop(columns='Ticket')
data = data.drop(columns='Fare')
null_all = data.isnull().sum()
null_all

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Embarked      2
dtype: int64

In [3]:
data.loc[:,"Age"] = data.fillna(data.loc[:,"Age"].median())
null_all = data.isnull().sum()
null_all

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Embarked    2
dtype: int64

In [4]:
data.iloc[:,2]

0        male
1      female
2      female
3      female
4        male
        ...  
886      male
887    female
888    female
889      male
890      male
Name: Sex, Length: 891, dtype: object

LabelEncoder


In [5]:
from sklearn.preprocessing import LabelEncoder

data.iloc[:,2] = LabelEncoder().fit_transform(data.iloc[:,2])
data.head()


,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,1,22.0,1,0,S
1,1,1,0,38.0,1,0,C
2,1,3,0,26.0,0,0,S
3,1,1,0,35.0,1,0,S
4,0,3,1,35.0,0,0,S


In [6]:
for i in data.columns:
    if i!= "Embarked":
        data[i] = (data[i] - min(data[i])) / (max(data[i])-min(data[i]))

In [7]:
new_data = data.copy()
def loaddata(data,label):
    datax = data[data[[label]].isnull().T.any()]
    for i in datax.index:
        data = data.drop(i)
    return data , datax




In [8]:
data , datax = loaddata(data,'Embarked')
ans = 0
datax.iloc[0,-1]

nan

In [9]:
def getNeighbors(trainingSet, testInstance, k):
    answer = {}
    for j in range(testInstance.shape[0]):
        distances = []
        distance0 = 0
        for x in range(trainingSet.shape[0]):
            for n in range(trainingSet.shape[1]):
                if n != trainingSet.shape[1]-1:
                    distance0 += (trainingSet.iloc[x,n]-testInstance.iloc[j,n])**2
            dist = (distance0)**0.5
            distances.append([trainingSet.iloc[x,:], dist])
        distances.sort(key = lambda x :x[-1])
        neighbors = []
        for x in range(k):
            neighbors.append(distances[x]) 
        classVotes = {}
        for i in range (len(neighbors)):
            response = neighbors [x][0][-1]
            if response in classVotes:
                classVotes[response] += 1
            else:
                classVotes[response] = 1
        sortedVotes = sorted(classVotes.items(), reverse=True)
        answer[j]=sortedVotes[0][0]
    return answer

In [10]:
neighbors=getNeighbors(data,datax,4)
neighbors[0]
for i in range(datax.shape[0]):
   datax.iloc[i,-1]=neighbors[i]
data = pd.concat([data,datax])
index01= np.arange(data.shape[0])
data.index = index01
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0.0,1.0,1.0,0.271174,0.125,0.000000,S
1,1.0,0.0,0.0,0.472229,0.125,0.000000,C
2,1.0,1.0,0.0,0.321438,0.000,0.000000,S
3,1.0,0.0,0.0,0.434531,0.125,0.000000,S
4,0.0,1.0,1.0,0.434531,0.000,0.000000,S
...,...,...,...,...,...,...,...
886,0.0,1.0,0.0,0.346569,0.125,0.333333,S
887,1.0,0.0,1.0,0.321438,0.000,0.000000,C
888,0.0,1.0,1.0,0.396833,0.000,0.000000,Q
889,1.0,0.0,0.0,0.472229,0.000,0.000000,S


In [11]:
x=data.iloc[:,[-1]]
x

,Embarked
0,S
1,C
2,S
3,S
4,S
...,...
886,S
887,C
888,Q
889,S


In [12]:
enc = OneHotEncoder(categories="auto").fit(x)
result = enc.transform(x).toarray()
result

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [13]:
pd.DataFrame(enc.inverse_transform(result))

,0
0,S
1,C
2,S
3,S
4,S
...,...
886,S
887,C
888,Q
889,S


In [14]:
enc.get_feature_names()

array(['x0_C', 'x0_Q', 'x0_S'], dtype=object)

In [15]:
new_data = pd.concat([data,pd.DataFrame(result)],axis=1)
new_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked,0,1,2
0,0.0,1.0,1.0,0.271174,0.125,0.0,S,0.0,0.0,1.0
1,1.0,0.0,0.0,0.472229,0.125,0.0,C,1.0,0.0,0.0
2,1.0,1.0,0.0,0.321438,0.000,0.0,S,0.0,0.0,1.0
3,1.0,0.0,0.0,0.434531,0.125,0.0,S,0.0,0.0,1.0
4,0.0,1.0,1.0,0.434531,0.000,0.0,S,0.0,0.0,1.0


In [16]:
new_data.drop(["Embarked"],axis=1,inplace=True)
new_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,0,1,2
0,0.0,1.0,1.0,0.271174,0.125,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.472229,0.125,0.0,1.0,0.0,0.0
2,1.0,1.0,0.0,0.321438,0.000,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.434531,0.125,0.0,0.0,0.0,1.0
4,0.0,1.0,1.0,0.434531,0.000,0.0,0.0,0.0,1.0


In [17]:
new_data.columns=['Survived','Pclass','Sex','Age','SibSp','Parch','Embark_C','Embark_Q','Embark_S']
new_data.head()
data = new_data

In [18]:

X=np.array(data.iloc[:,1:data.shape[1]])
#注意这里一定要将要y的形式变化成in32，不然无法进行筛选
y=np.array(data.iloc[:,0]).astype("int32")

y

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,

In [19]:
num_examples = len(X) # 训练样本的数量
nn_input_dim = data.shape[1] -1 # 输入层的维度
nn_output_dim = 2 # 输出层的维度
 
# 梯度下降的参数（我直接手动赋值）
epsilon = 0.01 # 梯度下降的学习率
reg_lambda = 0.01 # 正则化的强度

In [20]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [21]:
#帮助我们在数据集上估算总体损失的函数 
def calculate_loss(model):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    #正向传播，计算预测值
    X=np.array(data.iloc[:,1:data.shape[1]])
    y=np.array(data.iloc[:,0]).astype("int32")
    z1 = X.dot(W1) + b1
    a1 = sigmoid(z1)
    z2 = a1.dot(W2) + b2
    exp_scores = sigmoid(z2)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    # 计算损失
    corect_logprobs = -y.dot(np.log(probs[range(num_examples), y]))
    data_loss = np.sum(corect_logprobs)
    #在损失上加上正则项（可选）
    data_loss += reg_lambda/2 * (np.sum(np.square(W1)) + np.sum(np.square(W2)))
    return 1./num_examples * data_loss

In [22]:
# 预测输出（0或1）
def predict(model, x):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    # 正向传播
    z1 = x.dot(W1) + b1
    a1 = sigmoid(z1)
    z2 = a1.dot(W2) + b2
    exp_scores = np.exp(z2)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    return np.argmax(probs, axis=1)

In [23]:
def build_model(nn_hdim, num_passes=20000, print_loss=False):
     
    # 用随机值初始化参数。我们需要学习这些参数.
    W1 = np.random.randn(nn_input_dim, nn_hdim)*0.01
    b1 = np.zeros((1, nn_hdim))
    W2 = np.random.randn(nn_hdim, nn_output_dim)*0.01
    b2 = np.zeros((1, nn_output_dim))
 
    # 这是我们最终要返回的数据
    model = {}
     
    # 梯度下降 
    for i in range(0, num_passes):
 
        # 正向传播
        z1 = X.dot(W1) + b1
        a1 = sigmoid(z1)
        z2 = a1.dot(W2) + b2
        #softmax
        exp_scores = np.exp(z2)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
 
        # 反向传播
        delta3 = probs
        delta3[range(891), y] -= 1
        dW2 = (a1.T).dot(delta3)
        db2 = np.sum(delta3, axis=0, keepdims=True)
        delta2 = delta3.dot(W2.T) * (a1 -np.power(a1,2))
        dW1 = np.dot(X.T, delta2)
        db1 = np.sum(delta2, axis=0,keepdims=True)                    
 
        # 添加正则项 (b1 和 b2 没有正则项)
        dW2 += reg_lambda * W2
        dW1 += reg_lambda * W1
 
        # 梯度下降更新参数
        W1 += -epsilon * dW1
        b1 += -epsilon * db1
        W2 += -epsilon * dW2
        b2 += -epsilon * db2
         
        # 为模型分配新的参数
        model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
         
        # 选择性地打印损失
        if print_loss and i % 1000 == 0:
          print ("Loss after iteration %i: %f" %(i, calculate_loss(model)))
          
    return model

In [24]:
# 搭建一个3维隐藏层的模型
model = build_model(10, print_loss=True)
print(model)

Loss after iteration 0: 1.322900
Loss after iteration 1000: 0.395497
Loss after iteration 2000: 0.378128
Loss after iteration 3000: 0.365926
Loss after iteration 4000: 0.355241
Loss after iteration 5000: 0.351128
Loss after iteration 6000: 0.348679
Loss after iteration 7000: 0.346681
Loss after iteration 8000: 0.340535
Loss after iteration 9000: 0.320446
Loss after iteration 10000: 0.315846
Loss after iteration 11000: 0.314794
Loss after iteration 12000: 0.314204
Loss after iteration 13000: 0.313862
Loss after iteration 14000: 0.313643
Loss after iteration 15000: 0.313478
Loss after iteration 16000: 0.313332
Loss after iteration 17000: 0.313185
Loss after iteration 18000: 0.313023
Loss after iteration 19000: 0.312834
{'W1': array([[  0.89918496,   4.64008209,  -1.14164458,  -3.62967206,
        -12.02146247,  -2.46309334,  -9.74503175,  -0.14598972,
          8.17575724,  -8.14452792],
       [ -0.44565677,  -1.07477295,   6.9614149 ,  -7.61656728,
        -12.95958933,  -3.92581014,  